## initialization

In [1]:
import asyncio
from typing import Dict, List, Optional, Union, Callable
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.contrib.web_surfer import WebSurferAgent 
from autogen.agentchat.contrib.agent_builder import AgentBuilder
from autogen.formatting_utils import colored
from typing_extensions import Annotated
import autogen


from teachability import Teachability
from concurrent.futures import ThreadPoolExecutor, as_completed

import arxiv

import requests
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.pdf import partition_pdf

import os
import shutil
import requests
import pickle
import re
from pathlib import Path

import nest_asyncio
nest_asyncio.apply()

version = "0.0.1"
ProjectID = "FinNews"
initiate_db = True
config_file = "OAI_CONFIG_LIST"
agents_json = "./news-0.0.1.json"


Project_dir = Path(f"./{ProjectID}/{version}")

if not os.path.exists(Project_dir): initiate_db = True

output_dir = f'{Project_dir}/pdf_output'
if not os.path.exists(output_dir): os.makedirs(output_dir)

db_dir = f'{Project_dir}/memo-db/'
# check if db_dir exists, delete it if it does
if initiate_db:

    if not os.path.exists(Project_dir): 
        shutil.rmtree(Project_dir)
        os.makedirs(Project_dir)
    if os.path.exists(db_dir): shutil.rmtree(db_dir)

    # create a list of papers that have been read and saved it in a pickle file
    read_papers = []
    with open(f'{Project_dir}/read_papers.pkl', 'wb') as f:
        pickle.dump(read_papers, f)

    # create a list of abstract that have been read and saved it in a pickle file
    read_abstracts = []
    with open(f'{Project_dir}/read_abstracts.pkl', 'wb') as f:
        pickle.dump(read_abstracts, f)

config_list = autogen.config_list_from_json(
    config_file,
    file_location=".",
    filter_dict={
        "model": ["gpt-4-32k", "gpt-4", "gpt4", "gpt-35-turbo-16k", "gpt-4-0613", "gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613"]
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

# Configuration for the Language Model (LLM)
llm_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 120,
    # "seed": 42,
}

# Configuration for the manager using the same config_list as llm_config
manager_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 60,
    # "seed": 42,
}

# Termination message definition
termination_msg = (
    lambda x: isinstance(x, dict)
    and str(x.get("content", "")).upper() == "TERMINATE"
)

######################
bing_api_key = "bc0eaf95b2034706812cbc2ba03c1a99"

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        config_file,
        filter_dict={"model": ["gpt-4-32k", "gpt-35-turbo-16k"]},
    ),
    "temperature": 0,
}

print("Summarizer LLM models: ", [summarizer_llm_config["config_list"][i]["model"] for i in range(len(summarizer_llm_config["config_list"]))])

LLM models:  ['gpt-4', 'gpt-4-32k', 'gpt-4-0613', 'gpt-35-turbo', 'gpt-35-turbo-16k']
Summarizer LLM models:  ['gpt-4-32k', 'gpt-35-turbo-16k']


In [2]:
def create_teachable_groupchat(assitant_name, user_name, db_dir, config_list, verbosity=0):
    
    # Start by instantiating any agent that inherits from ConversableAgent.
    assistant = autogen.ConversableAgent(
        name=assitant_name,  # The name is flexible, but should not contain spaces to work in group chat.
        llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
    )

    # Instantiate the Teachability capability. Its parameters are all optional.
    teachability = Teachability(
        verbosity=verbosity,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
        reset_db=False,  
        path_to_db_dir=db_dir,
        recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
    )

    # Now add the Teachability capability to the agent.
    teachability.add_to_agent(assistant)

    user = autogen.UserProxyAgent(
        name=user_name,
        human_input_mode="NEVER",
        is_termination_msg=termination_msg,
        max_consecutive_auto_reply=0,
        code_execution_config={"use_docker": False},
    )

    return assistant, user

## Define the task

In [3]:
task = """
As a seasoned financial analysis writer, your expertise is called upon to compose an insightful weekly blog post that delves into the pivotal financial happenings from the preceding week. 
By web surfing, you MUST get the most up-to-date news. 
Your readers rely on your analytical prowess to unpack complex financial events and trends. 
Address the following crucial inquiries in your upcoming blog post to provide them with the clarity they seek:

Insights from recent central bank officials' statements, particularly from the Federal Reserve (Fed) and the European Central Bank (ECB).
Emerging trends in global monetary policy.
Key outcomes and implications from the latest Federal Reserve meeting.
Notable monetary policy meetings that occurred last week, with a focus on actions by the ECB, Fed, Bank of England (BoE), and Bank of Japan (BoJ).
A review of the latest inflation data from the Euro Area, the United Kingdom, Japan, and the United States.
Analysis of recent labor market reports from the USA and the Euro Area, including an examination of wage negotiations in Germany, France, Spain, and Italy.
Factors influencing the EUR/USD exchange rate fluctuations over the past week, including inflation data, fiscal budgets, and trade-related tariffs.
Developments in the pricing of inflation swaps in both the Eurozone and the United States.
Your discerning analysis is not just expected, but essential for readers who depend on your weekly posts to make informed financial decisions.
"""

## define function
### bing search

In [4]:
def memorize_web_search(search_results, query):

    # Create a TeachableAgent and UserProxyAgent to represent the researcher and the user, respectively.
    learner, learner_user = create_teachable_groupchat("learner", "learner_user", db_dir, config_list, verbosity=0)

    learner_user.initiate_chat(learner,
                       silent=True,
                       message=f"The following is the results of bing search for:'{query}': \n\n '{search_results}'")
    

def bing_search(query: str) -> str:
    # create websurfer agent
    web_surfer = WebSurferAgent(
        "web_surfer",
        llm_config=llm_config,
        summarizer_llm_config=summarizer_llm_config,
        browser_config={"viewport_size": 4096, "bing_api_key": bing_api_key},
    )

    user_proxy = autogen.UserProxyAgent(
        "user_proxy",
        human_input_mode="NEVER",
        code_execution_config=False,
        default_auto_reply="",
        is_termination_msg=lambda x: True,
        )
    
    res = user_proxy.initiate_chat(web_surfer, message=query)

    memorize_web_search(res, query)

def web_search(queries: Annotated[List[str], "liest of queries to search"]) -> str:
    
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(bing_search, query) for query in queries]
        for future in as_completed(futures):
            future.result() 

    # [bing_search(query) for query in queries]

    return f"The search results have been memorized. go ahead and accomplish your task: \n\n {task} \n\n if you need any help, let me know"

# web_search(["what is the best way to invest in stock market",
#             "how to invest in stock market",
#             ])


In [5]:
new_builder = AgentBuilder(config_file_or_env=config_file)
agent_list, agent_configs = new_builder.load(agents_json)

agents_dict = {}
for agent in agent_list:
    agents_dict[agent.name] = agent



# add teachablitity to "financial_news_research_analyst"

teachability = Teachability(
        verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
        reset_db=False,  
        path_to_db_dir=db_dir,
        recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
    )

teachability.add_to_agent(agents_dict["financial_news_research_analyst"])

# add web_search function to "financial_news_research_analyst"

for func, func_name, description in zip([web_search],
                                        ["web_search", ],
                                        ["Search the web for a list of queries."] ):
    
    for caller, executor in zip([agents_dict["financial_news_research_analyst"]],
                                [agents_dict["User_console_and_code_interpreter"]]):
        autogen.agentchat.register_function(
                func,
                caller=caller,
                executor=executor,
                name=func_name,
                description=description
            )


Loading config from ./news-0.0.1.json
==> Creating agents...
Creating agent financial_news_research_analyst with backbone gpt-4...
Creating agent central_banks_insights_specialist with backbone gpt-4...
Creating agent global_monetary_policy_analyst with backbone gpt-4...
Creating agent financial_data_analyst with backbone gpt-4...
Creating agent python_financial_news_aggregator_developer with backbone gpt-4...
Adding user console proxy...


In [6]:
def start_task(execution_task: str, agents: Dict[str, Union[str, List[str]]]):
    agent_list = list(agents.values())
    group_chat = autogen.GroupChat(agents=agent_list, 
                                   messages=[], 
                                   max_round=50) # type: ignore
    manager = autogen.GroupChatManager(groupchat=group_chat, llm_config={"config_list": config_list, **llm_config})
    agent_list[0].initiate_chat(manager,  # type: ignore
                                message=execution_task,
                                summary_method="reflection_with_llm"
                                )

In [7]:
# By employing the 'get_news' function with precise queries, you can access the most up-to-date news.

start_task(task, agents_dict)

User_console_and_code_interpreter (to chat_manager):


As a seasoned financial analysis writer, your expertise is called upon to compose an insightful weekly blog post that delves into the pivotal financial happenings from the preceding week. 
By web surfing, you MUST get the most up-to-date news. 
Your readers rely on your analytical prowess to unpack complex financial events and trends. 
Address the following crucial inquiries in your upcoming blog post to provide them with the clarity they seek:

Insights from recent central bank officials' statements, particularly from the Federal Reserve (Fed) and the European Central Bank (ECB).
Emerging trends in global monetary policy.
Key outcomes and implications from the latest Federal Reserve meeting.
Notable monetary policy meetings that occurred last week, with a focus on actions by the ECB, Fed, Bank of England (BoE), and Bank of Japan (BoJ).
A review of the latest inflation data from the Euro Area, the United Kingdom, Japan, and the Uni